# Package Installation and File reading

In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [43]:
import datetime
import time
from datetime import *

In [44]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [45]:
path = '/content/drive/Shareddrives/ISE 534/Project Solution/Data Manipulation/tps2_staff_pass.csv'
#path = '/content/drive/Shareddrives/ISE 534/Raw Data/data/tps2_0-10/tps2000000000001'

In [46]:
df1=pd.read_csv(path)
df1.head()

,Building,Level,ClientMacAddr,lat,lng,localtime,category
0,TPS2,Level 1,00:87:01:05:7a:bc,-22.815060,-43.245462,2019-08-21 09:53:23.601 UTC,passenger
1,TPS2,Level 1,28:39:5e:15:23:74,-22.814001,-43.247166,2019-08-21 09:53:23.932 UTC,staff
2,TPS2,Level 1,38:01:95:9a:0f:a4,-22.815099,-43.244897,2019-08-21 09:53:24.043 UTC,staff
3,TPS2,Level 1,d8:0d:17:1d:da:22,-22.815048,-43.246064,2019-08-21 09:53:24.047 UTC,staff
4,TPS2,Level 1,38:80:df:30:eb:08,-22.815112,-43.246586,2019-08-21 09:53:24.157 UTC,staff


# Geofencing on Zone 3

In [47]:
pip install geopandas

In [48]:
import geopandas as gpd

In [49]:
from shapely.geometry import Point, Polygon
import folium

In [50]:
#Function to create Polygon (in this case Zone 1 - square)
def create_polygon(coords, polygon_name):
 polygon = Polygon(coords)
 gdf = gpd.GeoDataFrame(crs = {"init" :"epsg:4326"})
 gdf.loc[0,"name"] = polygon_name
 gdf.loc[0, "geometry"] = polygon
 return gdf

In [51]:
#Function to determine if point is inside or outside
def geofencing_function(dataframe,zone_coordinates):
  # Convert Pandas Dataframe to GeoPandas Dataframe
  geo_df = gpd.GeoDataFrame(dataframe, geometry=gpd.points_from_xy(dataframe.lng, dataframe.lat))
  zone = create_polygon(zone_coordinates,"zone_x")


  # Functions to tell if point is inside or outside
  mask = (zone.loc[0, "geometry"])
  pip_mask_geofence = geo_df.within(mask)

  #Create PIP mask
  geo_df.loc[:,"geofence"] = pip_mask_geofence

  geo_df['geofence'] = geo_df['geofence'].replace({True: 'Inside', False: 'Outside'})
  return geo_df

In [52]:
coordinates_zone3 = [(-43.2474,-22.81474),(-43.24705, -22.81447), (-43.24663, -22.81538), (-43.24632, -22.81507)]

In [53]:
df1 = geofencing_function(df1, coordinates_zone3)

/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Assigning CRS to a GeoDataFrame without a geometry column is now deprecated and will not be supported in the future.
  after removing the cwd from sys.path.


In [54]:
df1['geofence'].value_counts()

Outside    12629304
Inside       340197
Name: geofence, dtype: int64

In [55]:
#Drop the geometry column and convert it back to a pandas dataframe
df1 = pd.DataFrame(df1.drop(columns='geometry'))

In [56]:
df1.head()

,Building,Level,ClientMacAddr,lat,lng,localtime,category,geofence
0,TPS2,Level 1,00:87:01:05:7a:bc,-22.815060,-43.245462,2019-08-21 09:53:23.601 UTC,passenger,Outside
1,TPS2,Level 1,28:39:5e:15:23:74,-22.814001,-43.247166,2019-08-21 09:53:23.932 UTC,staff,Outside
2,TPS2,Level 1,38:01:95:9a:0f:a4,-22.815099,-43.244897,2019-08-21 09:53:24.043 UTC,staff,Outside
3,TPS2,Level 1,d8:0d:17:1d:da:22,-22.815048,-43.246064,2019-08-21 09:53:24.047 UTC,staff,Outside
4,TPS2,Level 1,38:80:df:30:eb:08,-22.815112,-43.246586,2019-08-21 09:53:24.157 UTC,staff,Inside


In [57]:
#Filter out based on Geofencing = Inside
df1 = df1[df1['geofence'] == 'Inside']

# Subselect Day and Hour to analyze

In [58]:
df1['timestamp'] = df1['localtime'].str[:23]

In [59]:
df1['hour'] = df1['timestamp'].str[-12:]

In [60]:
df1['day'] = df1['timestamp'].str[:10]

In [61]:
df1.head()

,Building,Level,ClientMacAddr,lat,lng,localtime,category,geofence,timestamp,hour,day
4,TPS2,Level 1,38:80:df:30:eb:08,-22.815112,-43.246586,2019-08-21 09:53:24.157 UTC,staff,Inside,2019-08-21 09:53:24.157,09:53:24.157,2019-08-21
136,TPS2,Level 1,5c:03:39:0b:0f:27,-22.814976,-43.246778,2019-08-21 09:53:34.219 UTC,staff,Inside,2019-08-21 09:53:34.219,09:53:34.219,2019-08-21
275,TPS2,Level 1,e0:98:61:9a:0d:83,-22.814638,-43.247025,2019-08-21 09:53:45.927 UTC,staff,Inside,2019-08-21 09:53:45.927,09:53:45.927,2019-08-21
401,TPS2,Level 1,b8:27:eb:21:5e:9c,-22.814719,-43.246939,2019-08-21 09:53:55.721 UTC,staff,Inside,2019-08-21 09:53:55.721,09:53:55.721,2019-08-21
492,TPS2,Level 1,10:d0:7a:cb:1c:fb,-22.815255,-43.246617,2019-08-21 09:54:02.656 UTC,staff,Inside,2019-08-21 09:54:02.656,09:54:02.656,2019-08-21


In [62]:
#Subselect the day that we want to look at
day = df1[df1['day'] == "2019-08-21"]

In [63]:
#Subselect the specific time interval we want to look at
df_day_time = day[day["hour"].between('10:00:00','14:00:00')]

In [64]:
df_day_time.head()

,Building,Level,ClientMacAddr,lat,lng,localtime,category,geofence,timestamp,hour,day
4807,TPS2,Level 1,38:80:df:30:eb:08,-22.815181,-43.246620,2019-08-21 10:00:04.805 UTC,staff,Inside,2019-08-21 10:00:04.805,10:00:04.805,2019-08-21
4960,TPS2,Level 1,38:80:df:30:eb:08,-22.815216,-43.246580,2019-08-21 10:00:15.223 UTC,staff,Inside,2019-08-21 10:00:15.223,10:00:15.223,2019-08-21
4991,TPS2,Level 1,fc:b6:d8:a7:9c:a3,-22.815157,-43.246433,2019-08-21 10:00:17.34 UTC,passenger,Inside,2019-08-21 10:00:17.34,10:00:17.34,2019-08-21
4992,TPS2,Level 1,fc:b6:d8:a7:9c:a3,-22.815157,-43.246433,2019-08-21 10:00:17.341 UTC,passenger,Inside,2019-08-21 10:00:17.341,10:00:17.341,2019-08-21
5012,TPS2,Level 1,64:1c:ae:f9:79:da,-22.815232,-43.246557,2019-08-21 10:00:18.586 UTC,staff,Inside,2019-08-21 10:00:18.586,10:00:18.586,2019-08-21


In [65]:
#drop all the columns that we won't use
df_day_time.drop(columns='hour', inplace = True)
df_day_time.drop(columns='day', inplace = True)
df_day_time.drop(columns='timestamp', inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [66]:
df1 = df_day_time

# Main Code

# Data Cleaning and Convert Time to Date format

In [67]:
def data_clean(dataframe):
    dataframe.loc[:,'lat']=dataframe.loc[:,'lat'].astype("float64")
    dataframe.loc[:,'lng']=dataframe.loc[:,'lng'].astype("float64")
    dataframe[['date','time','formate']]=dataframe.localtime.str.split(' ', 0, expand=True)
    time=pd.to_datetime(dataframe.time,infer_datetime_format=True)
    dataframe.loc[:,'time']=time
    return(dataframe)

In [68]:
df1=data_clean(df1)


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


# Convert Date to timestamp(number) for calculation

In [69]:
def time_st(x):
    return(x.timestamp())

In [70]:
df1.loc[:,'time']=df1.time.map(time_st)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [71]:
df1

,Building,Level,ClientMacAddr,lat,lng,localtime,category,geofence,date,time,formate
4807,TPS2,Level 1,38:80:df:30:eb:08,-22.815181,-43.246620,2019-08-21 10:00:04.805 UTC,staff,Inside,2019-08-21,1.618826e+09,UTC
4960,TPS2,Level 1,38:80:df:30:eb:08,-22.815216,-43.246580,2019-08-21 10:00:15.223 UTC,staff,Inside,2019-08-21,1.618826e+09,UTC
4991,TPS2,Level 1,fc:b6:d8:a7:9c:a3,-22.815157,-43.246433,2019-08-21 10:00:17.34 UTC,passenger,Inside,2019-08-21,1.618826e+09,UTC
4992,TPS2,Level 1,fc:b6:d8:a7:9c:a3,-22.815157,-43.246433,2019-08-21 10:00:17.341 UTC,passenger,Inside,2019-08-21,1.618826e+09,UTC
5012,TPS2,Level 1,64:1c:ae:f9:79:da,-22.815232,-43.246557,2019-08-21 10:00:18.586 UTC,staff,Inside,2019-08-21,1.618826e+09,UTC
...,...,...,...,...,...,...,...,...,...,...,...
91707,TPS2,Level 1,d0:77:14:f2:42:7e,-22.815094,-43.246366,2019-08-21 13:59:36.875 UTC,passenger,Inside,2019-08-21,1.618841e+09,UTC
91732,TPS2,Level 1,7c:8b:b5:43:82:5d,-22.814559,-43.247020,2019-08-21 13:59:45.534 UTC,passenger,Inside,2019-08-21,1.618841e+09,UTC
91737,TPS2,Level 1,0c:9d:92:7c:19:29,-22.814634,-43.247179,2019-08-21 13:59:46.123 UTC,staff,Inside,2019-08-21,1.618841e+09,UTC
91749,TPS2,Level 1,38:9a:f6:53:67:af,-22.815025,-43.246783,2019-08-21 13:59:47.618 UTC,staff,Inside,2019-08-21,1.618841e+09,UTC


# Travel Time and distance calculation

In [72]:
def travel(dataframe):
    df=dataframe.copy()
    df.drop(columns=['formate'],inplace=True)
    df.sort_values(by=['Building','Level','ClientMacAddr','date','time'],inplace=True)
    
    #Calculate Travel Time in Seconds
    df.loc[:,'time']=df.time.diff()
    #Calculate Travel Distance in Meters
    df['Distance']=((69.5 * abs(df.lat - df.lat.shift(1)) + 57.3 * abs(df.lng - df.lng.shift(1)))*1609.344)
    
    df.fillna(0,inplace=True)
    df.index = range(df.shape[0])    
    
    #separate Different Mac Address
    df.loc[df[df.ClientMacAddr != df.ClientMacAddr.shift(1)].index,'Distance']=0
    df.loc[df[df.ClientMacAddr != df.ClientMacAddr.shift(1)].index,'time']=0
    
    #separate Different Date
    df.loc[df[df.date != df.date.shift(1)].index,'Distance']=0
    df.loc[df[df.date != df.date.shift(1)].index,'time']=0
    #separate Different Building
    df.loc[df[df.Building != df.Building.shift(1)].index,'Distance']=0
    df.loc[df[df.Building != df.Building.shift(1)].index,'time']=0
    #separate Different Level
    df.loc[df[df.Level != df.Level.shift(1)].index,'Distance']=0
    df.loc[df[df.Level != df.Level.shift(1)].index,'time']=0
    
    df['Speed_m/s']=df['Distance']/df['time']
    return(df)

Time is in sec. Distance is in meter

In [83]:
monitor=travel(df1)

In [84]:
monitor.head(10)

,Building,Level,ClientMacAddr,lat,lng,localtime,category,geofence,date,time,Distance,Speed_m/s
0,TPS2,Level 1,00:05:16:59:e4:bd,-22.814869,-43.246893,2019-08-21 10:00:31.137 UTC,staff,Inside,2019-08-21,0.000,0.000000,NaN
1,TPS2,Level 1,00:05:16:59:e4:bd,-22.815030,-43.246631,2019-08-21 10:09:15.777 UTC,staff,Inside,2019-08-21,524.640,42.144056,0.080329
2,TPS2,Level 1,00:05:16:59:e4:bd,-22.814678,-43.247015,2019-08-21 10:30:41.935 UTC,staff,Inside,2019-08-21,1286.158,74.773085,0.058137
3,TPS2,Level 1,00:05:16:59:e4:bd,-22.814834,-43.247009,2019-08-21 11:30:40.015 UTC,staff,Inside,2019-08-21,3598.080,18.074273,0.005023
4,TPS2,Level 1,00:05:16:59:e4:bd,-22.814881,-43.246914,2019-08-21 11:32:41.601 UTC,staff,Inside,2019-08-21,121.586,13.906607,0.114377
5,TPS2,Level 1,00:05:16:59:e4:bd,-22.814832,-43.246888,2019-08-21 11:50:38.798 UTC,staff,Inside,2019-08-21,1077.197,7.885789,0.007321
6,TPS2,Level 1,00:05:16:59:e4:bd,-22.814822,-43.247061,2019-08-21 13:00:22.416 UTC,staff,Inside,2019-08-21,4183.618,17.074985,0.004081
7,TPS2,Level 1,00:16:98:1c:9f:2b,-22.814712,-43.246984,2019-08-21 13:06:35.25 UTC,passenger,Inside,2019-08-21,0.000,0.000000,NaN
8,TPS2,Level 1,00:28:f8:4e:45:94,-22.815086,-43.246613,2019-08-21 13:03:35.564 UTC,passenger,Inside,2019-08-21,0.000,0.000000,NaN
9,TPS2,Level 1,00:3d:e8:af:bd:4f,-22.814839,-43.246922,2019-08-21 10:45:12.553 UTC,staff,Inside,2019-08-21,0.000,0.000000,NaN


In [107]:
#Devices travelling faster than 2 m/s
speedsters_df = monitor[monitor['Speed_m/s']>=2]
speedsters_df

,Building,Level,ClientMacAddr,lat,lng,localtime,category,geofence,date,time,Distance,Speed_m/s
24,TPS2,Level 1,00:9d:6b:31:9a:9e,-22.814958,-43.246735,2019-08-21 11:05:18.16 UTC,passenger,Inside,2019-08-21,10.197,36.357462,3.565506
62,TPS2,Level 1,00:ec:0a:7b:10:a0,-22.814739,-43.246927,2019-08-21 10:38:06.615 UTC,staff,Inside,2019-08-21,19.025,41.810558,2.197664
97,TPS2,Level 1,08:8c:2c:f9:57:9a,-22.815006,-43.246585,2019-08-21 10:12:48.468 UTC,passenger,Inside,2019-08-21,41.192,94.601994,2.296611
218,TPS2,Level 1,38:80:df:08:21:f8,-22.815078,-43.246410,2019-08-21 11:11:38.677 UTC,passenger,Inside,2019-08-21,11.563,32.005842,2.767953
239,TPS2,Level 1,38:80:df:30:eb:08,-22.815275,-43.246565,2019-08-21 10:00:49.151 UTC,staff,Inside,2019-08-21,7.792,15.849598,2.034086
246,TPS2,Level 1,38:80:df:30:eb:08,-22.815199,-43.246565,2019-08-21 10:04:14.838 UTC,staff,Inside,2019-08-21,10.577,28.047443,2.651739
265,TPS2,Level 1,38:80:df:30:eb:08,-22.815079,-43.246747,2019-08-21 10:11:46.04 UTC,staff,Inside,2019-08-21,10.896,35.523864,3.260267
267,TPS2,Level 1,38:80:df:30:eb:08,-22.815138,-43.246529,2019-08-21 10:11:56.742 UTC,staff,Inside,2019-08-21,10.701,26.772258,2.501846
486,TPS2,Level 1,48:49:c7:f9:bc:ee,-22.814990,-43.246674,2019-08-21 11:31:19.096 UTC,staff,Inside,2019-08-21,18.650,45.569793,2.443421
505,TPS2,Level 1,50:92:b9:b0:06:3e,-22.815120,-43.246608,2019-08-21 10:34:44.577 UTC,passenger,Inside,2019-08-21,31.073,112.869172,3.632387


In [89]:
# Calculating Average speed of movement
averages = monitor.groupby(['ClientMacAddr']).mean()

In [92]:
averages_list = list(averages['Speed_m/s'])
avg = np.nanmean(averages_list)
avg

0.2472080292285355

In [99]:
print('The average speed of movement within Zone 3 is', avg, 'm/s')

The average speed of movement within Zone 3 is 0.2472080292285355 m/s


In [100]:
monitor['Speed_m/s'].max()

3.9253221702631516

In [101]:
monitor['Speed_m/s'] = monitor['Speed_m/s'].fillna(0)
monitor.head()

,Building,Level,ClientMacAddr,lat,lng,localtime,category,geofence,date,time,Distance,Speed_m/s
0,TPS2,Level 1,00:05:16:59:e4:bd,-22.814869,-43.246893,2019-08-21 10:00:31.137 UTC,staff,Inside,2019-08-21,0.000,0.000000,0.000000
1,TPS2,Level 1,00:05:16:59:e4:bd,-22.815030,-43.246631,2019-08-21 10:09:15.777 UTC,staff,Inside,2019-08-21,524.640,42.144056,0.080329
2,TPS2,Level 1,00:05:16:59:e4:bd,-22.814678,-43.247015,2019-08-21 10:30:41.935 UTC,staff,Inside,2019-08-21,1286.158,74.773085,0.058137
3,TPS2,Level 1,00:05:16:59:e4:bd,-22.814834,-43.247009,2019-08-21 11:30:40.015 UTC,staff,Inside,2019-08-21,3598.080,18.074273,0.005023
4,TPS2,Level 1,00:05:16:59:e4:bd,-22.814881,-43.246914,2019-08-21 11:32:41.601 UTC,staff,Inside,2019-08-21,121.586,13.906607,0.114377


In [102]:
# Set up a customizable threshold. Let's say 50 can't be moving at a higher speed than 2 m/s
threshold_quant = 50
threshold_speed = 2

In [115]:
#aux = monitor[monitor['Speed_m/s']>=threshold_speed]
speedsters2 = list(speedsters_df['ClientMacAddr'].unique())

In [116]:
speedsters2

['00:9d:6b:31:9a:9e',
 '00:ec:0a:7b:10:a0',
 '08:8c:2c:f9:57:9a',
 '38:80:df:08:21:f8',
 '38:80:df:30:eb:08',
 '48:49:c7:f9:bc:ee',
 '50:92:b9:b0:06:3e',
 '50:bc:96:59:b7:ac',
 '50:bc:96:dc:27:35',
 '64:32:a8:1f:45:14',
 '6c:e8:5c:77:fc:ae',
 '70:ef:00:ef:9d:ed',
 '98:10:e8:de:48:bf',
 'b8:27:eb:21:5e:9c',
 'c0:9a:d0:a9:56:8d',
 'c0:9a:d0:f0:a1:86',
 'd4:dc:cd:e0:b1:94']

In [117]:
if len(speedsters2) >= threshold_quant:
  alarm = "Alert: There are too many people moving at a fast speed. Possible situation"
if len(speedsters2) < threshold_quant:
  alarm = "Everything is under control. People are moving at a normal speed"

In [118]:
print(alarm)

Everything is under control. People are moving at a normal speed
